## DM-48920

Eric Bellm & John Parejko, initially 2024-04-01 

In [8]:
import lsst.daf.butler

In [3]:
import lsst.afw.display
display1 = lsst.afw.display.Display(backend='firefly', frame=1)
display2 = lsst.afw.display.Display(backend='firefly', frame=2)

In [ ]:
(2024120600098, 2024112000208, 2024112800140, 2024111100094, 2024111900082, 2024111600297)

In [9]:

collection = 'LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810'
butler = lsst.daf.butler.Butler("/repo/main", collections=collection)

In [11]:
dataId = {'instrument': 'LSSTComCam', 'detector': 4, 'visit': 2024120600098}


In [37]:
template = butler.get('goodSeeingDiff_templateExp', dataId) 

display2.image(template)

In [15]:
patch_ids = butler.query_data_ids(['patch', 'band'], data_id=dataId, skymap='lsst_cells_v1') 
patch_ids

[{band: 'r', skymap: 'lsst_cells_v1', tract: 4848, patch: 80},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 4848, patch: 90},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 4848, patch: 91},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 4849, patch: 88},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 4849, patch: 89},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 4849, patch: 98},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 4849, patch: 99},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 5063, patch: 4},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 5063, patch: 5},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 5063, patch: 6},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 5063, patch: 14},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 5063, patch: 15},
 {band: 'r', skymap: 'lsst_cells_v1', tract: 5063, patch: 16}]

In [23]:
use_id = patch_ids[1]
print(use_id)
coadd = butler.get('goodSeeingCoadd', patch_ids[1])

{band: 'r', skymap: 'lsst_cells_v1', tract: 4848, patch: 90}


In [19]:
display1.image(coadd)

In [20]:
display3 = lsst.afw.display.Display(backend='firefly', frame=3)

In [21]:
deep = butler.get('deepCoadd', patch_ids[1])

In [22]:
display3.image(deep)

Initial inspection suggests that the NO_DATA patches in the template have corresponding UNMASKEDNAN mask planes in the goodSeeing and deep coadds.  This is despite the fact that the coadds have non-nan pixel values at those locations.  We hypothesize that these are detector-level features printing through to the coadd mask planes, hence the repetition at different translational and rotational dithers.

Apparent problems:

* unmasked nans at the detector level
* propagation of the UNMASKEDNAN mask plane into the coadd
* GetTemplateTask converting non-nan pixels with UNMASKEDNAN into NO_DATA (and setting their pixel values to nan!)
* in some cases GetTemplateTask sets the pixel values to nan but fails to set the NO_DATA mask plane.
  